In [1]:
import os

import bk.load
import bk.compute
import bk.plot
import bk.signal

import numpy as np

import scipy as sp


import matplotlib.pyplot as plt
import matplotlib as mpl
import neuroseries as nts

import pycorrelate

import basefunction.pdc_tdf as pdc_tdf

In [4]:
%matplotlib widget

In [3]:
path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
os.chdir(path)
bk.load.path = path
bk.load.rat = 8
bk.load.session = 'Rat08-20130713'
bk.load.day = 6

FileNotFoundError: [Errno 2] No such file or directory: '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'

In [5]:
states['sws'][0:1].as_units('s')

NameError: name 'states' is not defined

In [30]:
states = bk.load.states()
nrem_lfp = bk.load.lfp_in_intervals(166, 101, states['Rem'][0:1])

In [49]:
f, t, Sxx = sp.signal.spectrogram(nrem_lfp, 1250, nperseg=100, noverlap=75)

# plt.figure()
ax = plt.subplot(211)
plt.plot(nrem_lfp.as_units('s'))
plt.plot(bk.signal.passband(nrem_lfp, 40, 100).as_units('s'), 'r')
# bk.plot.intervals(states['sws'], 'grey')
plt.subplot(212, sharex=ax)
plt.pcolormesh(t+states['Rem'].as_units('s').iloc[0].start, f, Sxx)
plt.clim(0, 2000)
plt.ylim(0,100)

/home/billel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  if __name__ == '__main__':


(0.0, 100.0)

In [50]:
states['sws'].as_units('s')

,start,end
0,1448.0,1697.0
1,1777.0,1926.0
2,2104.0,2429.0
3,2826.0,3275.0
4,3383.0,3841.0
5,3888.0,3916.0
6,3970.0,3992.0
7,4024.0,4529.0
8,4568.0,4648.0
9,5044.0,5426.0
